In [3]:
import numpy as np
import pandas as pd
from pulp import *
import copy

<h2>Transition operator</h2>
<ul>
<li>In the precise case: $(\hat{T}^\dagger f)(s)  := \sum_{s'\in\mathcal{S}} T(s,s')  \cdot f(s')$
<li>In the imprecise case: $(\overline{\mathcal{T}}^{\dagger}f)(s):= \sup_{T(s,S') \in K(S'\mid s)} \sum_{\forall s'\in \mathcal{S}} T(s,s') \cdot f(s')$
    </ul>

In [4]:
def t(m, f, o='max'):
    d = len(m)
    output = np.zeros(d)
    if len(m.shape) == 3:  # Imprecise model
        for problem in range(d):
            model = LpProblem("Credal", LpMaximize if o == 'max' else LpMinimize)
            x = pulp.LpVariable.dicts('x', range(d))
            model += lpSum(x[i] * f[i] for i in range(d))  # Objective function
            model += lpSum(x[i] for i in range(d)) == 1.0
            for i in range(d):
                model += x[i] >= m[problem][0][i]
                model += x[i] <= m[problem][1][i]
            model.solve()
            output[problem] = model.objective.value()  # pulp.value(model.objective)
    else: # Precise model
        output = np.dot(m, f)
    return output

<h2>Markov Chain Object</h2>

In [43]:
class Chain:

    def __init__(self, name, t, states):
        self.name = name
        self.length = t
        self.states = states
        self.rewards = []
        self.T = []

    def set_t(self, m):
        self.T = m # Transition matrix
        
    def set_size(self, t):
        self.length = t # number of time steps
        
    def set_rew(self, v):
        self.rewards = np.array(v)
        
    def hitting(self, b, opt='max', verbose=False):
        results = pd.DataFrame(columns=self.states)
        one = np.ones(len(self.states))
        not_b = one - np.array(b)
        h0 = copy.deepcopy(b)
        for _ in range(self.length + 1):
            h1 = b + not_b * t(self.T, h0, opt)
            results.loc[_] = h1
            h0 = h1
        return results

    def conditional_hitting(self, phi1, phi2, opt='max'):
        results = pd.DataFrame(columns=self.states)
        one = np.ones(len(self.states))
        phi1_phi2 = phi1 * (one-phi2) # DEBUG
        h0 = copy.deepcopy(phi2)
        for _ in range(self.size + 1):
            h1 = phi2 + phi1_phi2 * t(self.T,  h0, opt)
            results.loc[_] = h1
            h0 = h1
        return results

    def cumulative(self, phi, opt='max'):
        results = pd.DataFrame(columns=self.states)
        one = np.ones(len(self.states))
        not_phi = one - np.array(phi)
        h0 = copy.deepcopy(self.rewards)
        for _ in range(self.size + 1):
            h1 = self.rewards + not_phi * t(self.T,  h0, opt)
            results.loc[_] = h1
            h0 = h1
        return results
    
    def cumulative_reward(self, b, opt='max'):
        results = pd.DataFrame(columns=self.states)
        one = np.ones(len(self.states))
        not_b = one - np.array(b)
        h0 = b * copy.deepcopy(self.rewards)  # initial exp rew DEBUG
        for _ in range(self.size + 1):
            h1 = one + not_b * t(self.T,  h0, opt)
            results.loc[_] = h1
            h0 = h1
        return results    

    def bounded_reward(self, phi1, phi2, rho, opt='max'):
        phi12 = np.array(phi1) * np.array(phi2)
        one = np.ones(len(self.states))
        phi12c = one - phi12 # DEBUG 
        x = np.zeros([self.size,rho,len(self.states)])
        for t in range(self.size):
            for r in range(rho):
                for s in range(len(self.states)):
                    if t == 0:
                        if self.rewards[s]<r & phi12[s] == 1:
                            x[t][r][s] = 1
                    else:
                        if phi12[s] == 1:
                            if self.rewards[s]<=r:
                                x[t][r][s] = 1
                            else:
                                x[t][r][s] = 0
        return x

Given a precise transition matrix compute an imprecise one by $\epsilon$-contamination and its logarithmic average

$$K_{\epsilon}(X) = \{ \epsilon P(X) + (1-\epsilon) P'(X) : \forall P'\}$$

In [44]:
def contaminate(m, eps):
    d = len(m)
    m2 = np.array([[[0. for _ in range(d)] for _ in range(2)] for _ in range(d)])
    for _ in range(d):
        if np.count_nonzero(m[_]) == 1:
            m2[_][0] = m[_]
            m2[_][1] = m[_]
        else:
            m2[_][0] = m[_]*(1.0-eps)
            m2[_][1] = [(0 if m[_][i] == 0 else m[_][i]*(1.0-eps) + eps) for i in range(d)]
    return m2

def average(ms,o='max'):
    if len(ms.shape) == 3:
        d = len(ms[0])
        m = np.ones([d,d])
        for k in range(d):
            tmp = np.ones(d)
            for mm in ms:
                tmp *= mm[k]
            tmp /= sum(tmp)
            m[k] = tmp
    else:
        d = ms.shape[1]
        m = np.array([[[0. for _ in range(d)] for _ in range(2)] for _ in range(d)])
        for k in range(d):
            l = np.ones(d)
            u = np.ones(d)
            l2 = np.ones(d)
            u2 = np.ones(d)
            for mm in ms:
                l *= mm[k][0]
                u *= mm[k][1]
            for i in range(d):
                l2[i] = l[i]/(sum(u)-u[i]+l[i])
                u2[i] = u[i]/(sum(l)-l[i]+u[i])
            m[k] = [l2,u2]
    return m

In [41]:
VERBOSE = True
LOGGING = True
mc = Chain("Two", 5, ["1", "2"])
transitions = np.array([[0.99, 0.01], [0.01, 0.99]])
rewards = [1,0]
mc.set_t(transitions)
mc.set_rew(rewards)
mc.bounded_reward([1,1],[1,1],3)
# mc.cumulative_reward([1,1])
# mc = Chain("Four", 10, ["S1", "S2", "S3","S4"])
# imc = Chain("Four2", 20, ["S1", "S2", "S3","S4"])
# transitions = np.array([[1.0, 0.0, 0.0, 0.0], [0.5, 0.0, 0.5, 0.0], [0.0, 0.5, 0.0, 0.5], [0.0, 0.0, 0.0, 1.0]])
# itransitions = contaminate(transitions,0.1)
# mc.set_t(transitions)
# imc.set_t(itransitions)
# print(mc.hitting([1, 0, 0, 0]))
# print(imc.hitting([1, 0, 0, 0],'min'))
# print(imc.hitting([1, 0, 0, 0],'max'))
# mc.set_rew([2,1,0,-10])
# mc.cumulative_reward([0,1,1,0])

T>0
T>0
T>0
T>0
[[[0. 1.]
  [0. 1.]
  [0. 1.]]

 [[0. 0.]
  [0. 0.]
  [0. 0.]]

 [[0. 0.]
  [0. 0.]
  [0. 0.]]

 [[0. 0.]
  [0. 0.]
  [0. 0.]]

 [[0. 0.]
  [0. 0.]
  [0. 0.]]]


In [64]:
# pmc = Chain("Messages", n, ["start", "delivery", "try", "lost"])
# transitions = np.array([[0, 0, 1, 0], [1, 0, 0, 0], [0, 0.9, 0, 0.1], [0, 0, 1, 0]])
# pmc.set_t(transitions)

# # Example 1 (Table 1)
# print(":: Table 1: Hitting Probs, index is t-1, each col is a different starting state")
# p = pmc.hitting([0, 0, 0, 1], verbose=True)

pmc2 = Chain("Messages", n, ["S1", "S2", "S3", "S4", "S5"])
transitions2 = np.array([
    [1.0, 0.0, 0.0, 0.0, 0.0], 
    [0.3, 0.0, 0.0, 0.7, 0.0], 
    [0.0, 1.0, 0.0, 0.0, 0.0], 
    [0.0, 0.0, 0.0, 0.0, 0.1],
    [0.0, 0.0, 0.0, 0.1, 0.0]])
pmc2.set_t(transitions2)
ppp = pmc2.hitting([1, 0, 1, 0, 0])
print(ppp)

pmc3 = Chain("Messages", n, ["S1", "S2", "S3", "S4"])
transitions3 = np.array([
    [1.0,  0.0, 0.0, 0.0], 
    [0.5,  0.0, 0.5, 0.0], 
    [0.0,  0.5, 0.0, 0.5],
    [0.0,  0.0, 0.0, 1.0]])
pmc3.set_t(transitions3)
ppp = pmc3.hitting([0, 0, 0, 1])
print(ppp)




# print(pmc.T)
# x0 = [1,0,0]
# x1 = np.dot(np.transpose(pmc.T),x0)
# x2 = np.dot(np.transpose(pmc.T),x1)
# print(x2[2])

# x0 = [1/3,1/3,1/3]
# x1 = np.dot(np.transpose(pmc.T),x0)
# print(x1)

# x0 = [1,0,0]
# x1 = np.dot(np.transpose(pmc.T),x0)
# x2 = np.dot(np.transpose(pmc.T),x1)
# print(x2)
# n = 10 
# pmc2 = Chain("Messages", n, ["S1", "S2", "S3", "S4"])
# transitions2 = np.array([[1/5, 1/5, 3/5, 2/5], [0,1,0,0], [0,1/2,0,1/2], [0,0,0,1]])
# pmc2.set_t((transitions2))
# p = pmc2.hitting([1, 0, 0, 0], verbose=False)
# print(p)

0.28
         S1        S2        S3
0  0.333333  0.333333  0.333333
1  0.333333  0.333333  0.333333
     S1    S2    S3
0  0.60  0.20  0.20
1  0.44  0.28  0.28
     S1   S2   S3   S4   S5
0   1.0  0.3  1.0  0.0  0.0
1   1.0  0.3  1.0  0.0  0.0
2   1.0  0.3  1.0  0.0  0.0
3   1.0  0.3  1.0  0.0  0.0
4   1.0  0.3  1.0  0.0  0.0
5   1.0  0.3  1.0  0.0  0.0
6   1.0  0.3  1.0  0.0  0.0
7   1.0  0.3  1.0  0.0  0.0
8   1.0  0.3  1.0  0.0  0.0
9   1.0  0.3  1.0  0.0  0.0
10  1.0  0.3  1.0  0.0  0.0
     S1        S2        S3   S4
0   0.0  0.000000  0.500000  1.0
1   0.0  0.250000  0.500000  1.0
2   0.0  0.250000  0.625000  1.0
3   0.0  0.312500  0.625000  1.0
4   0.0  0.312500  0.656250  1.0
5   0.0  0.328125  0.656250  1.0
6   0.0  0.328125  0.664062  1.0
7   0.0  0.332031  0.664062  1.0
8   0.0  0.332031  0.666016  1.0
9   0.0  0.333008  0.666016  1.0
10  0.0  0.333008  0.666504  1.0
